<h2>Proyecto: Detección de Lengua de Señas Mexicana utilizando Python</h2>
<p>Elaborado por:</p>
<ul>
    <li>Macias Ibarra Jonny</li>
    <li>López Gomez Hugo Octavio</li>
    <li>Ponce López José Eduardo</li>
    <li>Vieyra Perez Adonai</li>
</ul>
<hr>

<h2>Descripción</h2>
<p>El sistema captura video en tiempo real, extrae características de las extremidades superiores y clasifica gestos en letras del abecedario.Este código implementa un sistema de reconocimiento y clasificación de gestos manuales utilizando MediaPipe, OpenCV y un modelo de redes neuronales profundas entrenado en TensorFlow/Keras.</p>
<p>El código también permite la corrección manual de clasificaciones erróneas y el reentrenamiento del modelo con datos corregidos. Adicionalmente, captura fotogramas y calcula pendientes entre puntos clave.</p>
<hr>

<h2>Requerimientos</h2>
<p>Para ejecutar este código correctamente, se deben instalar las siguientes bibliotecas:</p>
<ul>
    <li><b>Python 3.10</b></li>
    <li><b>OpenCV (`cv2`):</b> Procesamiento de imágenes y captura de video.</li>
    <li><b>MediaPipe (`mediapipe`):</b> Detección de puntos clave del cuerpo y manos.</li>
    <li><b>TensorFlow (`tensorflow`):</b>Carga y ejecución del modelo de clasificación.</li>
    <li><b>Keras (`keras.preprocessing.image`):</b> Preprocesamiento de imágenes para el modelo.</li>
    <li><b>Scikit-learn (`sklearn.preprocessing.StandardScaler`):</b>Normalización de datos.</li>
    <li><b>Joblib (`joblib`):</b> Carga y almacenamiento del escalador de datos.</li>
    <li><b>Pandas (`pandas`):</b> Manejo de datos y almacenamiento en CSV.</li>
    <li><b>NumPy (`numpy`):</b>Manipulación de matrices numéricas.</li>
    <li><b>JSON (`json`):</b> Manejo de archivos en formato JSON.</li>
    <li><b>OS (`os`):</b> Manejo de archivos y rutas del sistema.</li>
</ul>
<p>Además, se requieren los siguientes archivos en los directorios espeificados:</p>
<ul>
    <li><b>Letras.h5</b>(Modelo entrenado en TensorFlow/Keras)</li>
    <li><b>escalador.pkl</b>(Escalador para normalización de datos)</li>
    <li><b>correcciones.csv</b>(Opcional: datos corregidos para reentrenamiento)</li>
</ul>
<p>El código debe ejecutarse en un entorno con acceso a una cámara para la captura en tiempo real.</p>
<hr>

<h2>Consideraciones del Modelo de Lenguaje</h2>
<p><b>Extracción de Características</b></p>
<ul>
    <li>Se extraen datos de los brazos y manos a partir de una estructura JSON con información de puntos clave.</li>
    <li>Las claves se ordenan para garantizar consistencia en la extracción.</li>
</ul>
<p><b>Clasificación</b></p>
<ul>
    <li>Se normalizan las características utilizando un escalador previamente entrenado.</li>
    <li>El modelo clasifica los gestos en una de las 26 letras del abecedario.</li>
    <li>Se descarta la predicción si la confianza es menor a 0.5.</li>
</ul>
<p><b>Correción y Reentrenamiento</b></p>
<ul>
    <li>Se pueden almacenar correcciones manuales en un archivo CSV.</li>
    <li>El modelo permite reentrenamiento con estos datos corregidos.</li>
    <li>Se utiliza categorical crossentropy y Adam optimizer para la actualización del modelo.</li>
</ul>
<p><b>Procesamiento en Tiempo Real</b></p>
<ul>
    <li>Se utiliza OpenCV para la captura de video y MediaPipe para el análisis del cuerpo</li>
    <li>Se capturan fotogramas a una tasa limitada para evitar sobrecarga de procesamiento.</li>
</ul>

<hr>
<h2>Resultados Esperados</h2>
<ul>
    <li>Predicción de una letra con una confianza asociada.</li>
    <li>Almacenamiento de datos corregidos en CSV.</li>
    <li>Posibilidad de reentrenamiento del modelo.</li>
</ul>

In [2]:
import cv2
import mediapipe as mp
import time
import json
import numpy as np
import tensorflow as tf
from keras.preprocessing import image
from sklearn.preprocessing import StandardScaler
import joblib
import pandas as pd
import os

In [3]:

# Variables globales 
"""
=====================================NOTA:====================================
Cambiar las direeciones de los archivos
==============================================================================

"""
model_path = "C:/Users/macia/Documents/Proyecto Mano/Deteccion_Cuerpo/Letras.h5" #Esta variable trae la red previamente entrenada
scaler_path = "escalador.pkl"
datos_correcciones_path = "correcciones.csv" # archuvo que nos va permitir reentrenar la red 
clases = ["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z"]
frames_data = []

model = tf.keras.models.load_model(model_path)
scaler = joblib.load(scaler_path)

mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic
mp_face_mesh = mp.solutions.face_mesh
FACE_CONNECTIONS = mp_face_mesh.FACEMESH_TESSELATION
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
mp_pose = mp.solutions.pose

start_time = time.time()
frame_counter = 0
last_capture_time = start_time

In [4]:
#devido a que guardamos en un json las pendientes, para tener mas oden 
#Aqui nos permitira extraer esa informacion para poder compararla con la de la red 
def extraccion(json_data):
    datos_brazo = json_data["datos_brazos"]
    caracteristicas = []

    caracteristicas.append(datos_brazo["Brazo Derecho"]["Brazo"])
    caracteristicas.append(datos_brazo["Brazo Derecho"]["Antebrazo"])

    caracteristicas.append(datos_brazo["Brazo Izquierdo"]["Brazo"])
    caracteristicas.append(datos_brazo["Brazo Izquierdo"]["Antebrazo"])

    for mano in ["Mano Derecha", "Mano Izquierda"]:
        claves_ordenadas = sorted(datos_brazo[mano].keys())
        for clave in claves_ordenadas:
            caracteristicas.append(datos_brazo[mano][clave])

    return np.array(caracteristicas)

In [5]:
#aqui pasamos un json que contiene las pendientes de los puntos 
#estos van a ser comparados con los de la red que estan en "Letas.h5" y en base a eso sacara una prediccion 
def clasificacion(json_input):
    try:
        x = extraccion(json_input)
        x = scaler.transform([x])
        predicciones = model.predict(x, verbose=0)
        clase_idx = np.argmax(predicciones)
        confianza = np.max(predicciones)

        if confianza < 0.5:
            return "Clase no determinada", confianza
        if clase_idx >= len(clases):
            return "Error: Clase desconocida", confianza

        return clases[clase_idx], confianza

    except Exception as e:
        print(f"Error en clasificación: {str(e)}")
        return "Error en procesamiento", 0.0


In [6]:
#Aqui guardamos los nuevos parametros en el archvo csv para en un futuro reentrenar la red 
def guardar_dato_correcto(json_input, clase_correcta):
    x = extraccion(json_input)
    df_nuevo = pd.DataFrame([x], columns=[f"feat_{i}" for i in range(len(x))])
    df_nuevo["clase"] = clase_correcta
    if os.path.exists(datos_correcciones_path):
        df_nuevo.to_csv(datos_correcciones_path, mode="a", header=False, index=False)
    else:
        df_nuevo.to_csv(datos_correcciones_path, index=False)

    print(f"Datos corregidos guardados para la clase {clase_correcta}")

In [7]:
#En dado caso de que la prdiccion que hace no es correcta, vamos a poder reentrenar la red para ajustar la prediccion 
def reentrenar_modelo():
    if not os.path.exists(datos_correcciones_path):
        print("No hay datos corregidos para reentrenar.")
        return
    df = pd.read_csv(datos_correcciones_path)
    X_train = df.iloc[:, :-1].values  # Todas las características
    y_train = df.iloc[:, -1].values  # Última columna (clase)

    # Convertir etiquetas a valores numéricos
    y_train = np.array([clases.index(c) for c in y_train])

    # Escalar características
    X_train = scaler.transform(X_train)

    # Convertir etiquetas a one-hot encoding
    y_train = tf.keras.utils.to_categorical(y_train, num_classes=len(clases))

    # Compilar el modelo con categorical_crossentropy
    model.compile(
        optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
    )

    # Reentrenar modelo
    model.fit(X_train, y_train, epochs=20, verbose=1)  # Ajustar epochs según necesidad

    # Guardar modelo actualizado
    model.save(model_path)
    print("Modelo reentrenado y guardado.")

In [8]:
#aqui calculamos las pendientes de los puntos x, y
def calcular_pendiente(punto1, punto2):
    x1, y1 = punto1
    x2, y2 = punto2
    if x2 - x1 == 0:  
        return float("inf") 
    return (y2 - y1) / (x2 - x1)

In [9]:
#Aqui mostramos la camara para predecir las señales
def mostrarCamara(prediccion, confianza, frame):
    cv2.putText(frame,f"{prediccion} ({confianza:.0%})",(10, 50),cv2.FONT_HERSHEY_SIMPLEX,1,(0, 255, 0),2,)
    cv2.imshow("Prediccion", frame)


In [10]:
#Con validacion nos ayudara ha decidir si la señal que estamos mostrando corresponde a lo que predice o no.
#Si lo que predice no es correcto, se guardaran en un archivo csv para reentrenar la red
def validacion(prediccion, frame_data):
     respuesta = input(f"¿Es correcta la predicción >>>>>>>>>'{prediccion}'? (s/n): ").strip().lower()
     if respuesta == 'n':
          clase_correcta = input("Introduce la letra correcta: ").strip().upper()
          if clase_correcta in clases:
               guardar_dato_correcto(frame_data, clase_correcta)
          else:
               print("Letra no válida.")
          return 0
     elif respuesta == 's':
          print("Predicción confirmada.")
          return 0
     else:
         return 27

In [11]:

#Este metodo nos va permitir usar la camara y dependiendo de la opcion elegida predecira o reentrenara
#Otra funsion que tiene es sacar las pendientes de los puntos del cuerpo, ya que no estamos comparando las imagenes, encambio 
#estamos usando las pendientes de los puntos para predecir
def camara(opc, last_capture_time):
     with mp_holistic.Holistic(static_image_mode=False, model_complexity=1) as holistic:
        while True:
            ret, frame = cap.read()
            if not ret:
                break

        # Calcular el tiempo transcurrido desde el último fotograma capturado
            current_time = time.time()
            elapsed_time = current_time - last_capture_time

        # Capturar solo 5 fotogramas por segundo
            if elapsed_time >= 0.2:  # 1 segundo / 5 = 0.2 segundos por fotograma
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = holistic.process(frame_rgb)

            # Diccionario para almacenar los datos del fotograma actual
                frame_data = {"id": len(frames_data) + 1, "datos_brazos": {}}

            # Postura (brazos)
                if results.pose_landmarks:
                    frame_data["datos_brazos"]["Brazo Derecho"] = {
                    "Brazo": calcular_pendiente(
                        (
                            results.pose_landmarks.landmark[11].x,
                            results.pose_landmarks.landmark[11].y,
                        ),
                        (
                            results.pose_landmarks.landmark[13].x,
                            results.pose_landmarks.landmark[13].y,
                        ),
                    ),
                    "Antebrazo": calcular_pendiente(
                        (
                            results.pose_landmarks.landmark[13].x,
                            results.pose_landmarks.landmark[13].y,
                        ),
                        (
                            results.pose_landmarks.landmark[15].x,
                            results.pose_landmarks.landmark[15].y,
                        ),
                    ),
                }
                    frame_data["datos_brazos"]["Brazo Izquierdo"] = {
                    "Brazo": calcular_pendiente(
                        (
                            results.pose_landmarks.landmark[12].x,
                            results.pose_landmarks.landmark[12].y,
                        ),
                        (
                            results.pose_landmarks.landmark[14].x,
                            results.pose_landmarks.landmark[14].y,
                        ),
                    ),
                    "Antebrazo": calcular_pendiente(
                        (
                            results.pose_landmarks.landmark[14].x,
                            results.pose_landmarks.landmark[14].y,
                        ),
                        (
                            results.pose_landmarks.landmark[16].x,
                            results.pose_landmarks.landmark[16].y,
                        ),
                    ),
                }
                else:
                    frame_data["datos_brazos"]["Brazo Derecho"] = {
                    "Brazo": 0,
                    "Antebrazo": 0,
                }
                    frame_data["datos_brazos"]["Brazo Izquierdo"] = {
                    "Brazo": 0,
                    "Antebrazo": 0,
                }

            # Mano derecha
                if results.right_hand_landmarks:
                    frame_data["datos_brazos"]["Mano Derecha"] = {
                    "0_1": calcular_pendiente(
                        (
                            results.right_hand_landmarks.landmark[0].x,
                            results.right_hand_landmarks.landmark[0].y,
                        ),
                        (
                            results.right_hand_landmarks.landmark[1].x,
                            results.right_hand_landmarks.landmark[1].y,
                        ),
                    ),
                    "1_2": calcular_pendiente(
                        (
                            results.right_hand_landmarks.landmark[1].x,
                            results.right_hand_landmarks.landmark[1].y,
                        ),
                        (
                            results.right_hand_landmarks.landmark[2].x,
                            results.right_hand_landmarks.landmark[2].y,
                        ),
                    ),
                    "2_3": calcular_pendiente(
                        (
                            results.right_hand_landmarks.landmark[2].x,
                            results.right_hand_landmarks.landmark[2].y,
                        ),
                        (
                            results.right_hand_landmarks.landmark[3].x,
                            results.right_hand_landmarks.landmark[3].y,
                        ),
                    ),
                    "3_4": calcular_pendiente(
                        (
                            results.right_hand_landmarks.landmark[3].x,
                            results.right_hand_landmarks.landmark[3].y,
                        ),
                        (
                            results.right_hand_landmarks.landmark[4].x,
                            results.right_hand_landmarks.landmark[4].y,
                        ),
                    ),
                    "0_5": calcular_pendiente(
                        (
                            results.right_hand_landmarks.landmark[0].x,
                            results.right_hand_landmarks.landmark[0].y,
                        ),
                        (
                            results.right_hand_landmarks.landmark[5].x,
                            results.right_hand_landmarks.landmark[5].y,
                        ),
                    ),
                    "5_6": calcular_pendiente(
                        (
                            results.right_hand_landmarks.landmark[5].x,
                            results.right_hand_landmarks.landmark[5].y,
                        ),
                        (
                            results.right_hand_landmarks.landmark[6].x,
                            results.right_hand_landmarks.landmark[6].y,
                        ),
                    ),
                    "6_7": calcular_pendiente(
                        (
                            results.right_hand_landmarks.landmark[6].x,
                            results.right_hand_landmarks.landmark[6].y,
                        ),
                        (
                            results.right_hand_landmarks.landmark[7].x,
                            results.right_hand_landmarks.landmark[7].y,
                        ),
                    ),
                    "7_8": calcular_pendiente(
                        (
                            results.right_hand_landmarks.landmark[7].x,
                            results.right_hand_landmarks.landmark[7].y,
                        ),
                        (
                            results.right_hand_landmarks.landmark[8].x,
                            results.right_hand_landmarks.landmark[8].y,
                        ),
                    ),
                    "5_9": calcular_pendiente(
                        (
                            results.right_hand_landmarks.landmark[5].x,
                            results.right_hand_landmarks.landmark[5].y,
                        ),
                        (
                            results.right_hand_landmarks.landmark[9].x,
                            results.right_hand_landmarks.landmark[9].y,
                        ),
                    ),
                    "9_10": calcular_pendiente(
                        (
                            results.right_hand_landmarks.landmark[9].x,
                            results.right_hand_landmarks.landmark[9].y,
                        ),
                        (
                            results.right_hand_landmarks.landmark[10].x,
                            results.right_hand_landmarks.landmark[10].y,
                        ),
                    ),
                    "10_11": calcular_pendiente(
                        (
                            results.right_hand_landmarks.landmark[10].x,
                            results.right_hand_landmarks.landmark[10].y,
                        ),
                        (
                            results.right_hand_landmarks.landmark[11].x,
                            results.right_hand_landmarks.landmark[11].y,
                        ),
                    ),
                    "11_12": calcular_pendiente(
                        (
                            results.right_hand_landmarks.landmark[11].x,
                            results.right_hand_landmarks.landmark[11].y,
                        ),
                        (
                            results.right_hand_landmarks.landmark[12].x,
                            results.right_hand_landmarks.landmark[12].y,
                        ),
                    ),
                    "9_13": calcular_pendiente(
                        (
                            results.right_hand_landmarks.landmark[9].x,
                            results.right_hand_landmarks.landmark[9].y,
                        ),
                        (
                            results.right_hand_landmarks.landmark[13].x,
                            results.right_hand_landmarks.landmark[13].y,
                        ),
                    ),
                    "13_14": calcular_pendiente(
                        (
                            results.right_hand_landmarks.landmark[13].x,
                            results.right_hand_landmarks.landmark[13].y,
                        ),
                        (
                            results.right_hand_landmarks.landmark[14].x,
                            results.right_hand_landmarks.landmark[14].y,
                        ),
                    ),
                    "14_15": calcular_pendiente(
                        (
                            results.right_hand_landmarks.landmark[14].x,
                            results.right_hand_landmarks.landmark[14].y,
                        ),
                        (
                            results.right_hand_landmarks.landmark[15].x,
                            results.right_hand_landmarks.landmark[15].y,
                        ),
                    ),
                    "15_16": calcular_pendiente(
                        (
                            results.right_hand_landmarks.landmark[15].x,
                            results.right_hand_landmarks.landmark[15].y,
                        ),
                        (
                            results.right_hand_landmarks.landmark[16].x,
                            results.right_hand_landmarks.landmark[16].y,
                        ),
                    ),
                    "13_17": calcular_pendiente(
                        (
                            results.right_hand_landmarks.landmark[13].x,
                            results.right_hand_landmarks.landmark[13].y,
                        ),
                        (
                            results.right_hand_landmarks.landmark[17].x,
                            results.right_hand_landmarks.landmark[17].y,
                        ),
                    ),
                    "0_17": calcular_pendiente(
                        (
                            results.right_hand_landmarks.landmark[0].x,
                            results.right_hand_landmarks.landmark[0].y,
                        ),
                        (
                            results.right_hand_landmarks.landmark[17].x,
                            results.right_hand_landmarks.landmark[17].y,
                        ),
                    ),
                    "17_18": calcular_pendiente(
                        (
                            results.right_hand_landmarks.landmark[17].x,
                            results.right_hand_landmarks.landmark[17].y,
                        ),
                        (
                            results.right_hand_landmarks.landmark[18].x,
                            results.right_hand_landmarks.landmark[18].y,
                        ),
                    ),
                    "18_19": calcular_pendiente(
                        (
                            results.right_hand_landmarks.landmark[18].x,
                            results.right_hand_landmarks.landmark[18].y,
                        ),
                        (
                            results.right_hand_landmarks.landmark[19].x,
                            results.right_hand_landmarks.landmark[19].y,
                        ),
                    ),
                    "19_20": calcular_pendiente(
                        (
                            results.right_hand_landmarks.landmark[19].x,
                            results.right_hand_landmarks.landmark[19].y,
                        ),
                        (
                            results.right_hand_landmarks.landmark[20].x,
                            results.right_hand_landmarks.landmark[20].y,
                        ),
                    ),
                }

                else:
                    frame_data["datos_brazos"]["Mano Derecha"] = {
                    "0_1": 0,
                    "1_2": 0,
                    "2_3": 0,
                    "3_4": 0,
                    "0_5": 0,
                    "5_6": 0,
                    "6_7": 0,
                    "7_8": 0,
                    "5_9": 0,
                    "9_10": 0,
                    "10_11": 0,
                    "11_12": 0,
                    "9_13": 0,
                    "13_14": 0,
                    "14_15": 0,
                    "15_16": 0,
                    "13_17": 0,
                    "0_17": 0,
                    "17_18": 0,
                    "18_19": 0,
                    "19_20": 0,
                }

            # Mano izquierda
                if results.left_hand_landmarks:
                    frame_data["datos_brazos"]["Mano Izquierda"] = {
                    "0_1": calcular_pendiente(
                        (
                            results.left_hand_landmarks.landmark[0].x,
                            results.left_hand_landmarks.landmark[0].y,
                        ),
                        (
                            results.left_hand_landmarks.landmark[1].x,
                            results.left_hand_landmarks.landmark[1].y,
                        ),
                    ),
                    "1_2": calcular_pendiente(
                        (
                            results.left_hand_landmarks.landmark[1].x,
                            results.left_hand_landmarks.landmark[1].y,
                        ),
                        (
                            results.left_hand_landmarks.landmark[2].x,
                            results.left_hand_landmarks.landmark[2].y,
                        ),
                    ),
                    "2_3": calcular_pendiente(
                        (
                            results.left_hand_landmarks.landmark[2].x,
                            results.left_hand_landmarks.landmark[2].y,
                        ),
                        (
                            results.left_hand_landmarks.landmark[3].x,
                            results.left_hand_landmarks.landmark[3].y,
                        ),
                    ),
                    "3_4": calcular_pendiente(
                        (
                            results.left_hand_landmarks.landmark[3].x,
                            results.left_hand_landmarks.landmark[3].y,
                        ),
                        (
                            results.left_hand_landmarks.landmark[4].x,
                            results.left_hand_landmarks.landmark[4].y,
                        ),
                    ),
                    "0_5": calcular_pendiente(
                        (
                            results.left_hand_landmarks.landmark[0].x,
                            results.left_hand_landmarks.landmark[0].y,
                        ),
                        (
                            results.left_hand_landmarks.landmark[5].x,
                            results.left_hand_landmarks.landmark[5].y,
                        ),
                    ),
                    "5_6": calcular_pendiente(
                        (
                            results.left_hand_landmarks.landmark[5].x,
                            results.left_hand_landmarks.landmark[5].y,
                        ),
                        (
                            results.left_hand_landmarks.landmark[6].x,
                            results.left_hand_landmarks.landmark[6].y,
                        ),
                    ),
                    "6_7": calcular_pendiente(
                        (
                            results.left_hand_landmarks.landmark[6].x,
                            results.left_hand_landmarks.landmark[6].y,
                        ),
                        (
                            results.left_hand_landmarks.landmark[7].x,
                            results.left_hand_landmarks.landmark[7].y,
                        ),
                    ),
                    "7_8": calcular_pendiente(
                        (
                            results.left_hand_landmarks.landmark[7].x,
                            results.left_hand_landmarks.landmark[7].y,
                        ),
                        (
                            results.left_hand_landmarks.landmark[8].x,
                            results.left_hand_landmarks.landmark[8].y,
                        ),
                    ),
                    "5_9": calcular_pendiente(
                        (
                            results.left_hand_landmarks.landmark[5].x,
                            results.left_hand_landmarks.landmark[5].y,
                        ),
                        (
                            results.left_hand_landmarks.landmark[9].x,
                            results.left_hand_landmarks.landmark[9].y,
                        ),
                    ),
                    "9_10": calcular_pendiente(
                        (
                            results.left_hand_landmarks.landmark[9].x,
                            results.left_hand_landmarks.landmark[9].y,
                        ),
                        (
                            results.left_hand_landmarks.landmark[10].x,
                            results.left_hand_landmarks.landmark[10].y,
                        ),
                    ),
                    "10_11": calcular_pendiente(
                        (
                            results.left_hand_landmarks.landmark[10].x,
                            results.left_hand_landmarks.landmark[10].y,
                        ),
                        (
                            results.left_hand_landmarks.landmark[11].x,
                            results.left_hand_landmarks.landmark[11].y,
                        ),
                    ),
                    "11_12": calcular_pendiente(
                        (
                            results.left_hand_landmarks.landmark[11].x,
                            results.left_hand_landmarks.landmark[11].y,
                        ),
                        (
                            results.left_hand_landmarks.landmark[12].x,
                            results.left_hand_landmarks.landmark[12].y,
                        ),
                    ),
                    "9_13": calcular_pendiente(
                        (
                            results.left_hand_landmarks.landmark[9].x,
                            results.left_hand_landmarks.landmark[9].y,
                        ),
                        (
                            results.left_hand_landmarks.landmark[13].x,
                            results.left_hand_landmarks.landmark[13].y,
                        ),
                    ),
                    "13_14": calcular_pendiente(
                        (
                            results.left_hand_landmarks.landmark[13].x,
                            results.left_hand_landmarks.landmark[13].y,
                        ),
                        (
                            results.left_hand_landmarks.landmark[14].x,
                            results.left_hand_landmarks.landmark[14].y,
                        ),
                    ),
                    "14_15": calcular_pendiente(
                        (
                            results.left_hand_landmarks.landmark[14].x,
                            results.left_hand_landmarks.landmark[14].y,
                        ),
                        (
                            results.left_hand_landmarks.landmark[15].x,
                            results.left_hand_landmarks.landmark[15].y,
                        ),
                    ),
                    "15_16": calcular_pendiente(
                        (
                            results.left_hand_landmarks.landmark[15].x,
                            results.left_hand_landmarks.landmark[15].y,
                        ),
                        (
                            results.left_hand_landmarks.landmark[16].x,
                            results.left_hand_landmarks.landmark[16].y,
                        ),
                    ),
                    "13_17": calcular_pendiente(
                        (
                            results.left_hand_landmarks.landmark[13].x,
                            results.left_hand_landmarks.landmark[13].y,
                        ),
                        (
                            results.left_hand_landmarks.landmark[17].x,
                            results.left_hand_landmarks.landmark[17].y,
                        ),
                    ),
                    "0_17": calcular_pendiente(
                        (
                            results.left_hand_landmarks.landmark[0].x,
                            results.left_hand_landmarks.landmark[0].y,
                        ),
                        (
                            results.left_hand_landmarks.landmark[17].x,
                            results.left_hand_landmarks.landmark[17].y,
                        ),
                    ),
                    "17_18": calcular_pendiente(
                        (
                            results.left_hand_landmarks.landmark[17].x,
                            results.left_hand_landmarks.landmark[17].y,
                        ),
                        (
                            results.left_hand_landmarks.landmark[18].x,
                            results.left_hand_landmarks.landmark[18].y,
                        ),
                    ),
                    "18_19": calcular_pendiente(
                        (
                            results.left_hand_landmarks.landmark[18].x,
                            results.left_hand_landmarks.landmark[18].y,
                        ),
                        (
                            results.left_hand_landmarks.landmark[19].x,
                            results.left_hand_landmarks.landmark[19].y,
                        ),
                    ),
                    "19_20": calcular_pendiente(
                        (
                            results.left_hand_landmarks.landmark[19].x,
                            results.left_hand_landmarks.landmark[19].y,
                        ),
                        (
                            results.left_hand_landmarks.landmark[20].x,
                            results.left_hand_landmarks.landmark[20].y,
                        ),
                    ),
                }
                else:
                    frame_data["datos_brazos"]["Mano Izquierda"] = {
                    "0_1": 0,
                    "1_2": 0,
                    "2_3": 0,
                    "3_4": 0,
                    "0_5": 0,
                    "5_6": 0,
                    "6_7": 0,
                    "7_8": 0,
                    "5_9": 0,
                    "9_10": 0,
                    "10_11": 0,
                    "11_12": 0,
                    "9_13": 0,
                    "13_14": 0,
                    "14_15": 0,
                    "15_16": 0,
                    "13_17": 0,
                    "0_17": 0,
                    "17_18": 0,
                    "18_19": 0,
                    "19_20": 0,
                }

            # Voltear el frame horizontalmente
                frame = cv2.flip(frame, 1)

            # Actualizar el tiempo del último fotograma capturado
                last_capture_time = current_time
                prediccion, confianza = clasificacion(frame_data)
                print(f"Predicción: {prediccion} (Confianza: {confianza:.2f})")
                key = 0
                #Aqui decidimos que se va hacer, predecir o reentrenar la REd
                if opc == 1:
                    mostrarCamara(prediccion, confianza, frame)
                else:
                   key = validacion(prediccion, frame_data)
                   if  key == 27:
                       if ( os.path.exists(datos_correcciones_path)and input("¿Reentrenar el modelo? (s/n): ").strip().lower() == "s"): reentrenar_modelo()

            if cv2.waitKey(1) & 0xFF == 27 or key == 27:
                break
     cap.release()
     cv2.destroyAllWindows()



In [12]:
if __name__ == '__main__':
    #Tenemos 2 funsiones 
    #opc 1 = nos va permitir usar la camara para predecir la señales
    #opc 0 = nos va permitir reentrenar la red, ya que pueda que no estre prediciendo de forma correcta 
    camara(1, last_capture_time)

Predicción: U (Confianza: 1.00)
Predicción: Clase no determinada (Confianza: 0.24)
Predicción: Clase no determinada (Confianza: 0.23)
Predicción: U (Confianza: 1.00)
Predicción: U (Confianza: 1.00)
Predicción: L (Confianza: 0.55)
Predicción: Clase no determinada (Confianza: 0.19)
Predicción: Clase no determinada (Confianza: 0.21)
Predicción: U (Confianza: 0.78)
Predicción: L (Confianza: 0.96)
Predicción: U (Confianza: 0.95)
Predicción: U (Confianza: 0.98)
Predicción: U (Confianza: 0.98)
Predicción: Clase no determinada (Confianza: 0.44)
Predicción: Clase no determinada (Confianza: 0.32)
Predicción: Clase no determinada (Confianza: 0.41)
Predicción: S (Confianza: 0.56)
Predicción: Clase no determinada (Confianza: 0.44)
Predicción: Clase no determinada (Confianza: 0.47)
Predicción: L (Confianza: 0.70)
Predicción: Clase no determinada (Confianza: 0.49)
Predicción: Clase no determinada (Confianza: 0.43)
Predicción: Clase no determinada (Confianza: 0.33)
Predicción: Clase no determinada (Co